## Leaving only mediode of each similarity cluster

In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

### Reading the input dataset

In [2]:
curr_dir = !pwd
pfam_version = "31"
input_path = curr_dir[0]+"/../../9.Features_exploration/features_tables_v"+pfam_version+"/"
filename = "windowed_positions_features_06.20.18.csv"

bind_scores_num = 10

#Features table
features_all = pd.read_csv(input_path+filename, sep='\t', index_col=0)
features_cols = features_all.columns[1:-bind_scores_num] #removing binding scores and domain name
ligands = ["dna", "dnabase", "dnabackbone", "rna", "rnabase", "rnabackbone", "peptide", "ion", "metabolite", "sm"]
print "all samples positions #: "+str(features_all.shape[0])

#similarity clusters
with open(curr_dir[0]+"/sim_clusters/pfam-v"+pfam_version+"/domain_to_cluster_dict.pik", 'rb') as handle:
    domain_to_cluster_dict = pickle.load(handle)
with open(curr_dir[0]+"/sim_clusters/pfam-v"+pfam_version+"/similarity_clusters_dict.pik", 'rb') as handle:
    similarity_clusters_dict = pickle.load(handle)
print "clusters #: "+str(len(similarity_clusters_dict.keys()))
print "domains in clusters #: "+str(len(domain_to_cluster_dict.keys()))

all samples positions #: 55936
clusters #: 36
domains in clusters #: 150


### In each cluster: leaving only the mediodes for the CV

In [3]:
#Finding which domains we need to remove
domains_for_removal = []
for domain in domain_to_cluster_dict.keys():
    cluster_num = domain_to_cluster_dict[domain]
    if (domain != similarity_clusters_dict[cluster_num]["mediode"][0]): #If there's more than one mediode, choosing one randomly
        domains_for_removal.append(domain)
print "domains to be removed #: "+str(len(domains_for_removal))

domains to be removed #: 114


In [4]:
#Finding all table indices (name of domain + position) for removal
table_idx_for_removal = []
for domain in domains_for_removal:
    domain_idx = features_all[features_all["domain_name"] == domain].index.tolist()
    table_idx_for_removal.extend(domain_idx)
print "total positions of removed domains #: "+str(len(table_idx_for_removal))

total positions of removed domains #: 13401


In [5]:
#Remove from the features table
features_all = features_all.drop(table_idx_for_removal)
print "all samples after mediodes filter #: "+str(features_all.shape[0])

all samples after mediodes filter #: 42535


In [6]:
#Saving to file for future runs
features_all.to_csv(curr_dir[0]+"/filtered_features_table/windowed_positions_features_mediode_filter_06.20.18.csv", sep='\t')